In [1]:
%pwd

'c:\\Users\\mahik\\Documents\\Git_serious\\Fine-tuning-text-summarizer---end-to-end-deployment\\research'

In [3]:
import os

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\mahik\\Documents\\Git_serious\\Fine-tuning-text-summarizer---end-to-end-deployment'

update config- entity :

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir : Path
    source_url : Path
    local_data_file : Path
    unzip_dir : Path

configuration manager : 

In [ ]:
# configuration manager
from src.Text_summarizer.constants import *
from src.Text_summarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        # root directory
        create_directories([config.root_dir]) 
        # return data ingestion config 
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config 

components:

In [ ]:
import os
import urllib.request as request
import zipfile
from src.Text_summarizer.logging import logger
from src.Text_summarizer.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} File is downloaded: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
# extracting zip file into artifacts folder

    def extract_zip_file(self):
    
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [ ]:
try : 
    config = ConfigurationManager()
    data_ingestion_config = config.get_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e